In [1]:
%load_ext watermark
%watermark

ModuleNotFoundError: No module named 'watermark'

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [4]:
from scipy.optimize import curve_fit

import tkinter as tk
from tkinter import filedialog

import os

from scipy import signal
import scipy.integrate as integrate

# %matplotlib qt

In [5]:
%run functions
matplotlib.rcParams.update({'font.size': 25})

In [6]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [7]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_path = filedialog.askopenfilename(multiple=True) # initialdir = dir_string
print(file_path)

time       = []
T_aux_avg  = []
r2_v2_rlim      = []

for i,j in enumerate(file_path):
    with load(j) as data:
        time.append(data['time'])
        T_aux_avg.append(data['temp'])
        r2_v2_rlim.append(data['r2_v2_rlim'])

('/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC10_RF10.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC11_RF11.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC12_RF12.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC13_RF13.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_tem

In [10]:
def tanh_to10(x,x0,K,theta):
    return 10**( K*(1+tanh(theta*(x-x0))) )
def tanh_plus_to10(x,x0,K,theta,a,b):
    return 10**( K*(1+tanh(theta*(x-x0))) + a*(x)+b )

def gen_logi_to10(x,A,K,C,B,M,nu):
    return 10**( A + ( K-A ) / ( C+exp(-B*(x-M)) )**(nu) )

def func2_to10(t, t0, A, B,  C, D):
    return 10**( A*(t-t0)/(C+(t-t0)**2)**D + B )
def func2abs_to10(t, t0, A, B,  C, D):
    return 10**( A*(t-t0)/(C+abs(t-t0))**D + B )
def func2ter_to10(t, t0, A, B, D):
    return 10**( A*(t-t0)**(2*D) / (1+(t-t0)**2)**D + B )
#def func2bis_to10(t, t0, A, B,  C, D):
#    tau = t-t0
#    return 10**( (A-B)*(tau) / (C+(tau**2)**(D/2))**(1/D) + B )
def func3_to10(t, t0, A, B,  C):
    return 10**( A*(t-t0) / (C+(t-t0)**2) + B )
def func4_to10(t, t0, A, B, C):
    return 10**( A*(t-t0) / (C+abs(t-t0)) + B )
def func5_to10(t, t0, A, B, C, k):
    return 10**( A*(t-t0) / (C+abs(t-t0)**k)**(1/k) + B )
def func6_to10(t, t0, A, C, k):
    return 10**( A*(t-t0) / (C+abs(t-t0)**k)**(1/k) )
def func7_to10(t, t0, A, B, k):
    return 10**( A*(t-t0) / (1+abs(t-t0)**k)**(1/k) + B)

In [28]:
shape(y)

(53465,)

In [31]:
# Temperature as a function of time
to_fit = 22 #11
x  = time[to_fit]
y  = T_aux_avg[to_fit]

sigma_size = r2_v2_rlim[to_fit][0][:len(y)] + r2_v2_rlim[to_fit][1][:len(y)] + r2_v2_rlim[to_fit][2][:len(y)]

figure('testfitfunctions',clear='True')
#xlim(0.8,6)

#plot(x*1e3, y ,label=r'$T$',color='C0')
b, a = signal.butter(8, 0.125)
y_filt = signal.filtfilt(b, a, y, padlen=150)
plot(x*1e3, y,label=r'$T filt$',color='C0')
plot(x*1e3, y_filt,label=r'$T filt$',color='C1',ls=':')


grid()
#hlines(y[],1000,x[-1],color='k',ls=':')
legend()
#xlim(0.8,x[end_fit]*1e3+0.1)
#xlim(13,15)
#ylim(-2,60)

In [32]:
# Size as a function of temperature
to_fit = 22 #11
x  = time[to_fit]
y  = T_aux_avg[to_fit]

sigma_size = r2_v2_rlim[to_fit][0][:len(y)] + r2_v2_rlim[to_fit][1][:len(y)] + r2_v2_rlim[to_fit][2][:len(y)]

figure('T fct size',clear='True')
#xlim(0.8,6)

#plot(x*1e3, y ,label=r'$T$',color='C0')
b, a = signal.butter(8, 0.125)
y_filt = signal.filtfilt(b, a, y, padlen=150)
plot(y,sigma_size,label=r'$T filt$',color='C0')
plot(y_filt,sigma_size,label=r'$T filt$',color='C1',ls=':')


grid()
#hlines(y[],1000,x[-1],color='k',ls=':')
legend()
#xlim(0.8,x[end_fit]*1e3+0.1)
#xlim(13,15)
#ylim(-2,60)

In [34]:
# Heat content as function of temperature
to_fit = 22 #11
x  = time[to_fit]
y  = T_aux_avg[to_fit]

sigma_size = r2_v2_rlim[to_fit][0][:len(y)] + r2_v2_rlim[to_fit][1][:len(y)] + r2_v2_rlim[to_fit][2][:len(y)]
R = r2_v2_rlim[to_fit][6][20]
V = 4/3*pi*R**3
rho = 256/V
aws = (1/rho)**(1/3)
temperature_gamma = C_e**2 / (4*pi*eps0*aws*kb*y)

figure('T fct size',clear='True')
#xlim(0.8,6)

#plot(x*1e3, y ,label=r'$T$',color='C0')
b, a = signal.butter(8, 0.125)
y_filt = signal.filtfilt(b, a, y, padlen=150)
# plot(y,label=r'$T filt$',color='C0')
plot(y_filt[:-1],diff(y_filt),label=r'$T filt$',color='C1',ls=':')


grid()
#hlines(y[],1000,x[-1],color='k',ls=':')
legend()
#xlim(0.8,x[end_fit]*1e3+0.1)
#xlim(13,15)
#ylim(-2,60)

In [64]:
for k in range(3):
    print(r2_v2_rlim[0][k+6][len(y)//100]*1000)

59.36495461747589
58.44011608904669
48.22632969977023


Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/usr/lib/python3.10/tkinter/__init__.py", line 839, in callit
    func(*args)
  File "/usr/lib/python3/dist-packages/matplotlib/backends/_backend_tk.py", line 489, in delayed_destroy
    self.window.destroy()
  File "/usr/lib/python3.10/tkinter/__init__.py", line 2341, in destroy
    self.tk.call('destroy', self._w)
_tkinter.TclError: can't invoke "destroy" command: application has been destroyed


In [48]:
for i,j in enumerate(file_path):
    print(i,j)

0 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz
1 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz
2 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz
3 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz
4 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz
5 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC10_RF10.npz
6 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC11_RF11.npz
7 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC12_RF12.npz
8 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC13_RF13.npz
9 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_

In [113]:
# OK pour la temperature
to_plot = 12
end_fit = len(time[to_plot])//8-150 # 125500

print(file_path[to_plot])
# test plot
index_max = 1900
figure(1,clear='True')
#semilogy(time[to_plot]*1e3,T_aux_avg[to_plot],color='r')
plot(time[to_plot]*1e3,T_aux_avg[to_plot],color='r')

#p0 = [1.8, 60, 63.45, 0.002,4e-3] # 124850
popt, pcov = curve_fit(tanh_plus_to10,
                       time[to_plot][:end_fit]*1e3, T_aux_avg[to_plot][:end_fit],
                       maxfev= 50000)
print(popt)
plot(time[to_plot][:end_fit]*1e3,tanh_plus_to10(time[to_plot][:end_fit]*1e3,*popt),
     color='k')

#plot(time[to_plot][end_fit:]*1e3,tanh_plus_to10(time[to_plot][end_fit:]*1e3,
#                                            1635.3,
#                                            14.075,
#                                            63.132,
#                                            0.87,
#                                            -3324.94),
#     color='g',ls=':')


#p0 = [5e-3,500,1,2,65,2] # 124850
popt, pcov = curve_fit(gen_logi_to10,
                       time[to_plot][:end_fit]*1e3, T_aux_avg[to_plot][:end_fit],
                       maxfev= 50000)
print(popt)
plot(time[to_plot][:end_fit]*1e3,gen_logi_to10(time[to_plot][:end_fit]*1e3,*popt),
     color='xkcd:azul')

p0 = [1.95*1e-3,log10(500)/2,0.1,1e-1,0.3]
popt, pcov = curve_fit(func5, x[:end_fit], log10(y[:end_fit]), p0, maxfev= 50000)
print(popt)
y_fit = func5(x[:end_fit],*popt)
plot(x[:end_fit]*1e3,10**y_fit,color='g')
plot(x[:end_fit]*1e3,10**func5(x[:end_fit],1.95*1e-3,log10(500),0.1,1e-1,0.3),
     color='xkcd:rouge')

vlines(time[to_plot][end_fit]*1e3,0,50)
#b, a = signal.butter(8, 0.125)
#y = signal.filtfilt(b, a, T_aux_avg[to_plot], padlen=150)
#semilogy(time[to_plot]*1e3,y,color='b')
xlabel('time [ms]')
ylabel('T [K]')
#xlim(61,64)
#ylim(-1,20)
grid()
tight_layout()

/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC09_RF09.npz


/usr/lib/python3/dist-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


[ 2.25086968e+03  1.02252835e+01  1.53877294e+00  7.22290265e-01
 -4.50178924e+03]
[-6.96613793e+02  1.46465340e+01  1.00072493e+00  7.34778003e+00
  6.18398243e-01  2.53304942e+01]
[ 2.40563243e-03  1.28719328e+01 -1.71381882e+00  2.14636270e-01
  2.69872890e-01]


/tmp/ipykernel_90434/3493093275.py:16: RuntimeWarning: invalid value encountered in power
  return A*(t-t0) / (C+abs(t-t0)**k)**(1/k) + B


In [629]:
t0_first_guess,A0,B0

(0.013997624500000002, 41.99381961231696, 0.517953442987538)

In [630]:
to_fit = 22 #11
start_for_fit = 1000
end_for_fit = [int(500000-1),140000,17700,7000,4800, # 17700
               470000,480000,490000,27500,5500, # 24000
               50000,28500,23500,30000,14500,
              0,0,0,0,0,
              0,0,0,0,0,
              0,28500]
end_fit = end_for_fit[to_fit]
x  = time[to_fit]
y  = T_aux_avg[to_fit]
T_crit = 0.5
t0_first_guess = x[where(min(abs(y-T_crit))==abs(y-T_crit))][0] # 2.5e-3
A0 = log10(max(y[start_for_fit:end_fit]))
B0 = log10(y[where(min(abs(y-T_crit))==abs(y-T_crit))][0])
figure('testfitinlog',clear='True')
#xlim(0.8,6)
grid()
semilogy(x*1e3, y ,label=r'$T$',color='C0')
b, a = signal.butter(8, 0.125)
y_filt = signal.filtfilt(b, a, y, padlen=150)
semilogy(x*1e3, y_filt ,label=r'$T filt$',color='C1',ls=':')
vlines(x[end_fit]*1e3,1e-2,150,color='k')

p0 = [t0_first_guess,A0,B0,0.001]
popt, pcov = curve_fit(func4, x[start_for_fit:end_fit], log10(y[start_for_fit:end_fit]), p0,  maxfev= 50000)
y_fit4 = func4(x[start_for_fit:end_fit],*popt)
print('func4',popt)
semilogy(x[start_for_fit:end_fit]*1e3,10**y_fit4,color='r',ls=':',label=r'$|x|$')


p0 = [t0_first_guess,A0,B0,0.05,0.05]
popt, pcov = curve_fit(func5, x[start_for_fit:end_fit], log10(y[start_for_fit:end_fit]), p0, maxfev= 50000)
print('func5',popt)
kfunc5 = popt[4]
y_fit5 = func5(x[start_for_fit:end_fit],*popt)
semilogy(x[start_for_fit:end_fit]*1e3,10**y_fit5,color='r',ls='--',label=r'$|x|^k$')

p0 = [t0_first_guess,A0,B0,0.05,0.4]
popt, pcov = curve_fit(func2, x[start_for_fit:end_fit], log10(y[start_for_fit:end_fit]), p0, maxfev= 50000)
print('func2',popt)
y_fit2 = func2(x[start_for_fit:end_fit],*popt)
semilogy(x[start_for_fit:end_fit]*1e3,10**y_fit2,color='m',label=r'$(t-t_0)^2$')

p0 = [t0_first_guess,A0,B0,0.05**kfunc5,kfunc5*2]
popt, pcov = curve_fit(func2bis, x[start_for_fit:end_fit], log10(y[start_for_fit:end_fit]), p0, maxfev= 50000)
print('func2bis',popt)
kfunc2bis = popt[4]
y_fit2bis = func2bis(x[start_for_fit:end_fit],*popt)
semilogy(x[start_for_fit:end_fit]*1e3,10**y_fit2bis,color='orange',label=r'$(t-t_0)^{2D/2}$')

p0 = [t0_first_guess,A0,100,100,0]
popt, pcov = curve_fit(tanh_plus, x[start_for_fit:end_fit], log10(y[start_for_fit:end_fit]), p0, maxfev= 50000)
print('tanh_plus',popt)
y_fittanh = tanh_plus(x[start_for_fit:end_fit],*popt)
semilogy(x[start_for_fit:end_fit]*1e3,10**y_fittanh,color='g',ls='-',label=r'tanh')

#hlines(y[],1000,x[-1],color='k',ls=':')
legend()
xlim(0.8,x[end_fit]*1e3+0.1)
#xlim(1.7,2.3)
ylim(8e-4,120)

func4 [1.86812450e-03 4.92689014e+02 5.13132937e-01 1.00000000e-03]


/tmp/ipykernel_90434/817932116.py:15: RuntimeWarning: overflow encountered in power
  return 10**( A*(t-t0) / (C+abs(t-t0)) + B )
/usr/lib/python3/dist-packages/scipy/optimize/_minpack_py.py:792: RuntimeWarning: overflow encountered in square
  cost = np.sum(infodict['fvec'] ** 2)
/usr/lib/python3/dist-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/tmp/ipykernel_90434/803482994.py:27: RuntimeWarning: overflow encountered in power
  plot(x[start_for_fit:end_fit]*1e3,10**y_fit4,color='r',ls=':',label=r'$|x|$')


"\np0 = [t0_first_guess,A0,B0,0.05,0.05]\npopt, pcov = curve_fit(func5, x[start_for_fit:end_fit], log10(y[start_for_fit:end_fit]), p0, maxfev= 50000)\nprint('func5',popt)\nkfunc5 = popt[4]\ny_fit5 = func5(x[start_for_fit:end_fit],*popt)\nsemilogy(x[start_for_fit:end_fit]*1e3,10**y_fit5,color='r',ls='--',label=r'$|x|^k$')\n\np0 = [t0_first_guess,A0,B0,0.05,0.4]\npopt, pcov = curve_fit(func2, x[start_for_fit:end_fit], log10(y[start_for_fit:end_fit]), p0, maxfev= 50000)\nprint('func2',popt)\ny_fit2 = func2(x[start_for_fit:end_fit],*popt)\nsemilogy(x[start_for_fit:end_fit]*1e3,10**y_fit2,color='m',label=r'$(t-t_0)^2$')\n\np0 = [t0_first_guess,A0,B0,0.05**kfunc5,kfunc5*2]\npopt, pcov = curve_fit(func2bis, x[start_for_fit:end_fit], log10(y[start_for_fit:end_fit]), p0, maxfev= 50000)\nprint('func2bis',popt)\nkfunc2bis = popt[4]\ny_fit2bis = func2bis(x[start_for_fit:end_fit],*popt)\nsemilogy(x[start_for_fit:end_fit]*1e3,10**y_fit2bis,color='orange',label=r'$(t-t_0)^{2D/2}$')\n\np0 = [t0_first_

In [391]:
to_fit = 22 #11
start_for_fit = 1000
end_for_fit = [int(500000-1),140000,17700,7000,4800, # 17700
               470000,480000,490000,27500,5500, # 24000
               50000,28500,23500,30000,14500,
              0,0,0,0,0,
              0,0,27000,0,0,
              0,0,0,0,0]
end_fit = end_for_fit[to_fit]
x  = time[to_fit]
y  = T_aux_avg[to_fit]
T_crit = 0.5
t0_first_guess = x[where(min(abs(y-T_crit))==abs(y-T_crit))][0] # 2.5e-3
B0 = log10(T_crit) # log10( y[where(min(abs(y-T_crit))==abs(y-T_crit))][0] )
A0 = log10( max(y[start_for_fit:end_fit]) ) - B0
figure('testfitinlin',clear='True')
#xlim(0.8,6)
grid()
plot(x*1e3, y ,label=r'$T$',color='C0')
b, a = signal.butter(8, 0.125)
y_filt = signal.filtfilt(b, a, y, padlen=150)
plot(x*1e3, y_filt ,label=r'$T filt$',color='C1',ls=':')
vlines(x[end_fit]*1e3,1e-2,150,color='k')

p0 = [t0_first_guess,A0,B0,0.05,0.05]
popt, pcov = curve_fit(func5_to10, x[start_for_fit:end_fit], y[start_for_fit:end_fit], p0, maxfev= 50000)
print('func5',popt)
kfunc5 = popt[4]
y_fit5 = func5_to10(x[start_for_fit:end_fit],*popt)
plot(x[start_for_fit:end_fit]*1e3,y_fit5,color='r',ls='--',label=r'5$|t-t_0|^k$')

#p0 = [0.01400588, 1.98778611,0, 0.00722586, 0.39195976]
#plot(x[start_for_fit:end_fit]*1e3,func5_to10(x[start_for_fit:end_fit],*p0),color='r',ls=':',label=r'$|t-t_0|^k$')

p0 = [t0_first_guess,log10(120),7e-3,0.35]
popt, pcov = curve_fit(func6_to10, x[start_for_fit:end_fit], y[start_for_fit:end_fit], p0, maxfev= 50000)
print('func6',popt)
kfunc6 = popt[3]
y_fit6 = func6_to10(x[start_for_fit:end_fit],*popt)
plot(x[start_for_fit:end_fit]*1e3,y_fit6,color='r',ls='--',label=r'6$|t-t_0|^k$')

#p0 = [t0_first_guess,1000,log10(9e-3),1e-5]
#plot(x[start_for_fit:end_fit]*1e3,func7_to10(x[start_for_fit:end_fit],*p0),color='chartreuse',ls='--',label=r'$|t-t_0|^k C=1$')

p0 = [t0_first_guess,  A0, B0, 1e-10, 0.45]
#p0 = [t0_first_guess,A0,B0,0.05,0.4]
popt, pcov = curve_fit(func2_to10, x[start_for_fit:end_fit], y[start_for_fit:end_fit], p0, maxfev= 100000)
print('func2',popt)
y_fit2 = func2_to10(x[start_for_fit:end_fit],*popt)
plot(x[start_for_fit:end_fit]*1e3,y_fit2,color='m',label=r'2$(t-t_0)^2$')

p0 = [t0_first_guess,  A0, B0, 1e-10, 0.45]
#p0 = [t0_first_guess,A0,B0,0.05,0.4]
popt, pcov = curve_fit(func2abs_to10, x[start_for_fit:end_fit], y[start_for_fit:end_fit], p0, maxfev= 100000)
print('func2abs',popt)
y_fit2abs = func2abs_to10(x[start_for_fit:end_fit],*popt)
plot(x[start_for_fit:end_fit]*1e3,y_fit2abs,color='m',ls=':',label=r'2abs$(t-t_0)^2$')

'''
p0 = [t0_first_guess,A0,B0,0.05**kfunc5,kfunc5*2]
popt, pcov = curve_fit(func2bis_to10, x[start_for_fit:end_fit], y[start_for_fit:end_fit], p0, maxfev= 50000)
print('func2bis',popt)
kfunc2bis = popt[4]
y_fit2bis = func2bis_to10(x[start_for_fit:end_fit],*popt)
plot(x[start_for_fit:end_fit]*1e3,y_fit2bis,color='deeppink',label=r'2b$(t-t_0)^2D/2$')

p0 = [t0_first_guess,log10(200),1e3]
popt, pcov = curve_fit(tanh_to10, x[start_for_fit:end_fit], y[start_for_fit:end_fit], p0, maxfev= 50000)
print('func2bis',popt)
kfunc2tanh = popt[2]
y_fittanh = tanh_to10(x[start_for_fit:end_fit],*popt)
plot(x[start_for_fit:end_fit]*1e3,y_fittanh,color='deeppink',label=r'$\tanh$')

p0 = [t0_first_guess,A0,B0,0.001]
popt, pcov = curve_fit(func4_to10, x[start_for_fit:end_fit], y[start_for_fit:end_fit], p0,  maxfev= 50000)
y_fit4 = func4_to10(x[start_for_fit:end_fit],*popt)
print('func4',popt)
plot(x[start_for_fit:end_fit]*1e3,y_fit4,color='r',ls=':',label=r'$|x|$')

p0 = [t0_first_guess,1000,log10(9e-3),1e-5]
popt, pcov = curve_fit(func7_to10, x[start_for_fit:end_fit], y[start_for_fit:end_fit], p0, maxfev= 50000)
print('func7',popt)
kfunc7 = popt[3]
y_fit7 = func7_to10(x[start_for_fit:end_fit],*popt)
plot(x[start_for_fit:end_fit]*1e3,y_fit7,color='chartreuse',label=r'7$|t-t_0|^k$, $C=1$')

'''

'''
p0 = [t0_first_guess,A0,100,100,0]
popt, pcov = curve_fit(tanh_plus_to10, x[start_for_fit:end_fit], y[start_for_fit:end_fit], p0, maxfev= 50000)
print('tanh_plus',popt)
y_fittanh = tanh_plus_to10(x[start_for_fit:end_fit],*popt)
plot(x[start_for_fit:end_fit]*1e3,10**y_fittanh,color='g',ls='-',label=r'tanh')
'''

#hlines(y[],1000,x[-1],color='k',ls=':')
legend()
#xlim(0.8,x[end_fit]*1e3+0.1)
xlim(10,15)
ylim(-1,80)

/tmp/ipykernel_184041/4030997247.py:23: RuntimeWarning: invalid value encountered in power
  return 10**( A*(t-t0) / (C+abs(t-t0)**k)**(1/k) + B )


func5 [ 1.39996885e-02  3.63325811e+02 -3.60601330e+02  1.38183167e-04
  1.87281447e-01]
func6 [0.01400588 2.26778114 0.00722573 0.39196203]
func2 [ 1.39675873e-02  1.30272867e+01 -9.50820729e+00  2.40880072e-10
  4.89555670e-01]
func2abs [ 1.39976245e-02  1.92953921e+00 -3.01029996e-01  1.00000000e-10
  4.50000000e-01]


/tmp/ipykernel_184041/4030997247.py:12: RuntimeWarning: invalid value encountered in power
  return 10**( A*(t-t0)/(C+abs(t-t0))**D + B )
/tmp/ipykernel_184041/4030997247.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return 10**( A*(t-t0)/(C+abs(t-t0))**D + B )
/tmp/ipykernel_184041/4030997247.py:12: RuntimeWarning: overflow encountered in true_divide
  return 10**( A*(t-t0)/(C+abs(t-t0))**D + B )
/tmp/ipykernel_184041/4030997247.py:12: RuntimeWarning: overflow encountered in power
  return 10**( A*(t-t0)/(C+abs(t-t0))**D + B )


(0.0008, 80.0)

In [387]:
10**(-9.50820729e+00)

3.1030781265267713e-10

In [321]:
figure('displaylogfit',clear='True')
#xlim(0.8,6)
grid()
#semilogy(x*1e3, y ,label=r'$T$',color='C0')
plot(x*1e3, log(y) ,label=r'$T filt$',color='C0',lw=0.5)
#plot(x*1e3, arctanh(y_filt) ,label=r'$T filt$',color='C1',ls=':')
#semilogy(x[start_for_fit:end_fit]*1e3,y_fit6,color='r',ls='--',label=r'$|x|^k$')
#semilogy(x[start_for_fit:end_fit]*1e3,y_fit2,color='m',label=r'$(t-t_0)^2$')

In [369]:
xtest=linspace(0,50,10000)
t0test=2
figure('testplot',clear='True')
#xlim(0.8,6)
grid()
#plot(x*1e3, y ,label=r'$T$',color='C0')
#plot(x*1e3, y_filt ,label=r'$T filt$',color='C1',ls=':')

B = -0.28 #log10(0.1)
A = 4 - B
print(A,B)
p0 = [ 14,  log10(100), log10(1e-1),  1, 0.45]
#plot(xtest,func5(xtest,*p0),color='r')
plot(xtest,func2_to10(xtest,*p0),color='b')

print(min(func2_to10(xtest,*p0)))
tight_layout()

4.28 -0.28
0.000252316507639044


In [435]:
# OK pour la temperature
x = linspace(-10,10,1000)

print(file_path[to_plot])
# test plot
index_max = 1900
figure(1111,clear='True')
#semilogy(time[to_plot]*1e3,T_aux_avg[to_plot],color='r')
plot(x,tanh_plus(x,1,1,0,0.1,0),color='r')
#b, a = signal.butter(8, 0.125)
#y = signal.filtfilt(b, a, T_aux_avg[to_plot], padlen=150)
#semilogy(time[to_plot]*1e3,y,color='b')
xlabel('time [ms]')
ylabel('T [K]')
# xlim(1,1.1)
grid()
tight_layout()

/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC09_RF09.npz


In [495]:
N_period

24.60000000000015

In [241]:
time_step = 48//4 # 488 # 488
y_filt_diff = y_filt[time_step:]-y_filt[:-time_step]
x_avg = (x[:-time_step]) *1e3 # x_avg = (x[time_step:]+x[:-time_step])/2 *1e3
N_period = (x[time_step]-x[0])*(Omega/2/pi)

figure(11,clear='True')
plot(x_avg,y_filt_diff/N_period)
plot(x[start_for_fit:end_fit-1]*1e3,
     diff(y_fit2)/(diff(x[start_for_fit:end_fit])*(Omega/2/pi)),color='m' ) # / 10000000
plot(x[start_for_fit:end_fit-1]*1e3,
     diff(y_fit2bis)/(diff(x[start_for_fit:end_fit])*(Omega/2/pi)),color='deeppink' ) # / 10000000
plot(x[start_for_fit:end_fit-1]*1e3,
     diff(y_fit5)/(diff(x[start_for_fit:end_fit])*(Omega/2/pi)),color='r',ls='--' ) # / 10000000
plot(x[start_for_fit:end_fit-1]*1e3,
     diff(y_fit6)/(diff(x[start_for_fit:end_fit])*(Omega/2/pi)),color='r',ls=':') # / 10000000

xlabel('t [ms]')
ylabel('H [K/period]')
grid()
xlim(t0_first_guess*1e3-1,t0_first_guess*1e3+1)
ylim(-1,1)
tight_layout()

In [232]:
y_fit7

array([1.04030154, 1.04030154, 1.04030154, ..., 1.04030154, 1.04030154,
       1.04030154])